Importo las librerías necesarias

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import pyarrow
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler


Traigo el dataset

In [126]:
df_test = pd.read_parquet('test.parquet')
df_test.head()

,id,url,region,region_url,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7037609789,https://annarbor.craigslist.org/apa/d/wixom-ho...,ann arbor,https://annarbor.craigslist.org,manufactured,1344,3,2.0,0,0,...,0,0,0,w/d in unit,off-street parking,https://images.craigslist.org/00M0M_iNczP1nzIL...,"OPEN HOUSE TODAY! APPLY THIS WEEK, PUT A HOLDI...",42.5333,-83.5763,mi
1,7032406876,https://vermont.craigslist.org/apa/d/randolph-...,vermont,https://vermont.craigslist.org,apartment,1050,2,1.0,0,0,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00L0L_ecirmYBIzL...,"Think of it, you'll be first to get your mail....",43.9393,-72.5538,vt
2,7037022682,https://annarbor.craigslist.org/apa/d/ann-arbo...,ann arbor,https://annarbor.craigslist.org,apartment,1150,2,2.0,1,1,...,1,0,0,w/d in unit,carport,https://images.craigslist.org/00e0e_dPln2xjo9g...,One of Ann Arbor's most luxurious apartment co...,42.2492,-83.7712,mi
3,7048681802,https://fortcollins.craigslist.org/apa/d/fort-...,fort collins / north CO,https://fortcollins.craigslist.org,apartment,1280,2,2.5,1,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00L0L_jlektT5cSd...,"Specials! Move in before January 16th, 2020 an...",40.5501,-105.0350,co
4,7043597870,https://charlottesville.craigslist.org/apa/d/c...,charlottesville,https://charlottesville.craigslist.org,apartment,783,2,1.0,1,1,...,0,0,0,laundry on site,None,https://images.craigslist.org/00D0D_cXa4KbZ6ox...,Barracks West Apartments & Townhomes in Charlo...,38.0936,-78.5611,va


In [127]:
df_test.columns

Index(['id', 'url', 'region', 'region_url', 'type', 'sqfeet', 'beds', 'baths',
       'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'laundry_options',
       'parking_options', 'image_url', 'description', 'lat', 'long', 'state'],
      dtype='object')

Saco las columnas que no voy a usar

In [128]:
df_test.drop(['id', 'url', 'region', 'region_url','beds', 'baths',
       'cats_allowed','smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'laundry_options',
       'parking_options', 'image_url', 'description', 'lat', 'long', 'state', 'dogs_allowed'], axis=1, inplace=True)

Primera mirada a la data

In [129]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38498 entries, 0 to 38497
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    38498 non-null  object
 1   sqfeet  38498 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 601.7+ KB


Genero dummies de la columna type para poder usarla en mi modelo

In [130]:
df_dummies = pd.get_dummies(df_test, columns=['type'])

In [131]:
df_dummies.head()

,sqfeet,type_apartment,type_condo,type_cottage/cabin,type_duplex,type_flat,type_house,type_in-law,type_loft,type_manufactured,type_townhouse
0,1344,0,0,0,0,0,0,0,0,1,0
1,1050,1,0,0,0,0,0,0,0,0,0
2,1150,1,0,0,0,0,0,0,0,0,0
3,1280,1,0,0,0,0,0,0,0,0,0
4,783,1,0,0,0,0,0,0,0,0,0


Escalo el dataset.

In [133]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_dummies)

Inicio la función kmeans y entreno el dataset escalado

In [135]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(data_scaled)

genero labels para poder luego contrastar la data

In [136]:
klabels = kmeans.labels_
klabels

array([0, 0, 0, ..., 0, 2, 0], dtype=int32)

calculo la inercia

In [137]:
kmeans.inertia_

325741.14571642014

Hago la predicción

In [139]:

pred = kmeans.predict(data_scaled)

Calculo el silhouette score, que indica cuán bien se ajusta cada valor a su cluster en comparación con otros clusters

In [140]:
print(silhouette_score(data_scaled, kmeans.labels_))

0.6929825239719593


In [141]:
len(pred)

38498

In [142]:
monicav84 = pd.DataFrame(data=pred, columns=['pred']).set_index('pred')
monicav84

""
pred
0
0
0
0
0
...
0
0
0


In [143]:
monicav84.to_csv('monicav84.csv')